# Environement Setup

In [ ]:
# ATTENZIONE: eseguire solo su Colab!
import os
import sys

# --- 1. SETUP AMBIENTE E DRIVE ---

from google.colab import drive
from google.colab import userdata

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# --- 2. CONFIGURAZIONE GITHUB (PRIVATO) ---
# Sostituisci QUESTE variabili con i tuoi dati reali
GITHUB_USER = 'MarcoPernoVDP'
REPO_NAME = 'MistakeDetection'
BRANCH = 'main'

# Recupera il Token dai Segreti di Colab ("GITHUB_TOKEN")
try:
    GIT_TOKEN = userdata.get('GITHUB_TOKEN')
except ImportError:
    # Se non usi i Segreti, incolla il token qui (SCONSIGLIATO ma funziona)
    GIT_TOKEN = "INCOLLA_QUI_IL_TUO_PAT_SE_NON_USI_SEGRETI"
except Exception:
    print("⚠️ ERRORE: Non ho trovato il segreto 'GITHUB_TOKEN'.")
    print("Vai sulla chiave inglese a sinistra -> Aggiungi nuovo segreto -> Nome: GITHUB_TOKEN")
    raise

REPO_URL = f'https://{GIT_TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git'
ROOT_DIR = f'/content/{REPO_NAME}'

# --- 3. CLONE O AGGIORNAMENTO CODICE ---
if not os.path.exists(ROOT_DIR):
    print(f"🔄 Clonazione del repository {REPO_NAME}...")
    !git clone {REPO_URL}
else:
    print(f"🔄 Aggiornamento repository (git pull)...")
    %cd {ROOT_DIR}
    !git pull origin {BRANCH}
    %cd /content

# --- 4. CONFIGURAZIONE PATH PYTHON ---
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)
    print(f"✅ Aggiunto {ROOT_DIR} al path di sistema.")

# --- 5. INSTALLAZIONE DIPENDENZE ---

#

print("\nSETUP COMPLETATO")

# Dataset Setup

In [2]:
import os
import sys

# Rileva se siamo su Google Colab
try:
    import google.colab
    ON_COLAB = True
except ImportError:
    ON_COLAB = False

if ON_COLAB:
    print("Esecuzione su Google Colab")
    project_root = "/content/MistakeDetection"
    if project_root not in sys.path:
        sys.path.append(project_root)
    from utils.setup_dataset import setup_dataset
    data_source_path = os.path.join("/content", "drive", "MyDrive", "AML_MistakeDetection_DATA")
else:
    print("Esecuzione su ambiente locale (VSCode)")
    project_root = os.path.abspath("..")
    if project_root not in sys.path:
        sys.path.append(project_root)
    from utils.setup_dataset import setup_dataset
    data_source_path = os.path.join(project_root, "_file")

# Chiama la funzione
setup_dataset(data_source_path, project_root)

print(f"Dati estratti e organizzati in: {os.path.join(project_root, 'data')}")


Esecuzione su ambiente locale (VSCode)
Cartella 'annotation_json' copiata in 'c:\Users\dylan\Desktop\Università\5° ANNO\AML\MistakeDetection\data/'.
File ZIP 'c:\Users\dylan\Desktop\Università\5° ANNO\AML\MistakeDetection\_file\omnivore_test.zip' estratto in 'c:\Users\dylan\Desktop\Università\5° ANNO\AML\MistakeDetection\data/'.
Dati estratti e organizzati in: c:\Users\dylan\Desktop\Università\5° ANNO\AML\MistakeDetection\data


In [3]:
import os
import sys
import torch
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

# Root del progetto
project_root = os.path.abspath("..") # serve per far si che Python veda il percorso della cartella padre "MistakeDetection" per poter importare i moduli delle altre cartelle
if project_root not in sys.path:
    sys.path.append(project_root)

from dataset.loader import CaptainCook4D_Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device in uso:", device)

Device in uso: cpu


In [5]:
if ON_COLAB:
    # Definisci la root del progetto sul Drive
    project_root = '/content/MistakeDetection'
    # Percorsi dati
    features_dir = os.path.join(project_root, "data", "omnivore_test")
    annotations_dir = os.path.join(project_root, "data", "annotation_json")
else:
    features_dir = os.path.join("..", "data", "omnivore_test")          # cartella npz
    annotations_dir = os.path.join("..", "data", "annotation_json")  # cartella JSON

# --- Inizializza il dataset completo ---
full_dataset = CaptainCook4D_Dataset(features_dir, annotations_dir)

# --- Indici di train, val e test ---
indices = list(range(len(full_dataset)))

# 20% test
trainval_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42, shuffle=True)
# 10% validation sul rimanente
val_relative_size = 0.1 / 0.8
train_idx, val_idx = train_test_split(trainval_idx, test_size=val_relative_size, random_state=42, shuffle=True)

# --- Creazione dei Subset ---
train_dataset = Subset(full_dataset, train_idx)
val_dataset   = Subset(full_dataset, val_idx)
test_dataset  = Subset(full_dataset, test_idx)

# --- Creazione DataLoader ---
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# --- Controllo dimensioni ---
print(f"Full Dataset samples (100%): {len(full_dataset)}\n")
print(f"Train samples (70%): {len(train_dataset)}")
print(f"Validation samples (10%): {len(val_dataset)}")
print(f"Test samples (20%): {len(test_dataset)}")


Full Dataset samples (100%): 9798

Train samples (70%): 6858
Validation samples (10%): 980
Test samples (20%): 1960


In [ ]:
print(f"FULL DATASET shape: {full_dataset.shape()}")  # Output leggibile
print(f"Numero di campioni: {full_dataset.shape()[0]}")
print(f"Numero di feature: {full_dataset.shape()[1]}")


def count_labels(dataset, name="Dataset"):
    cnt_0, cnt_1 = 0, 0
    for i in range(len(dataset)):
        label = dataset[i][1].item()
        if label == 0:
            cnt_0 += 1
        else:
            cnt_1 += 1
    print(f"\n{name}:")
    print(f"label_0 = {cnt_0}")
    print(f"label_1 = {cnt_1}")
    print(f"label_1/label_0 = {cnt_1/cnt_0:.4f}")
    print(f"label_0/label_1 = {cnt_0/cnt_1:.4f}")
    return cnt_0, cnt_1

# Conta le label per ciascun subset
train_cnt_0, train_cnt_1 = count_labels(train_dataset, "TRAIN DATASET")
val_cnt_0, val_cnt_1     = count_labels(val_dataset, "VALIDATION DATASET")
test_cnt_0, test_cnt_1   = count_labels(test_dataset, "TEST DATASET")

In [ ]:
import numpy as np
import os

def inspect_npz(npz_path, n_rows=5):
    """
    Mostra il contenuto di un file .npz.

    Args:
        npz_path (str): percorso del file .npz
        n_rows (int): numero di righe da stampare per ogni array
    """
    if not os.path.exists(npz_path):
        print(f"[ERROR] File non trovato: {npz_path}")
        return

    data = np.load(npz_path)
    print(f"File: {npz_path}")
    print("Chiavi presenti nel file:", list(data.keys()))

    for key in data.keys():
        arr = data[key]
        print(f"\nArray '{key}' - shape: {arr.shape}, dtype: {arr.dtype}")
        print(arr[:n_rows])  # stampa le prime n_rows righe

# Esempio di utilizzo
inspect_npz("../data/omnivore_test/1_7_360p.mp4_1s_1s.npz")


# MLP (Version 1)

In [ ]:
import torch.nn as nn
from models.MLP_version1 import MLP_version1

model = MLP_version1(1024).to(device)

In [ ]:
lr = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr)

# Quanto pesa la classe “positiva” = classe "1" = classe "error":
# - CASO 1: rapporto effettivo del dataset
#train_pos_weight = train_cnt_0 / train_cnt_1

# - CASO 2: rapporto usato nel paper
train_pos_weight = 1.5

print(f"Peso classe positiva: {train_pos_weight}")
train_pos_weight = torch.tensor([train_pos_weight], device=device)

criterion = nn.BCEWithLogitsLoss(pos_weight=train_pos_weight)

epochs = 50

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

for epoch in range(epochs):

    # -------------------------
    #        TRAIN
    # -------------------------
    model.train()
    total_loss = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        outputs = model(inputs)            # [B, 1]
        outputs = outputs.squeeze(1)       # [B]

        loss = criterion(outputs, labels)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)

    # -------------------------
    #        EVAL
    # -------------------------
    model.eval()
    total_val_loss = 0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device).float()

            outputs = model(inputs).squeeze(1)  # logits

            # same loss as train
            val_loss = criterion(outputs, labels)
            total_val_loss += val_loss.item()

            # convert logits → probabilities → binary predictions
            probs = torch.sigmoid(outputs)
            preds = (probs >= 0.5).long()

            all_preds.append(preds.cpu())
            all_targets.append(labels.cpu())

    # concat
    all_preds = torch.cat(all_preds).numpy()
    all_targets = torch.cat(all_targets).numpy()

    avg_val_loss = total_val_loss / len(test_loader)
    acc = accuracy_score(all_targets, all_preds)
    f1  = f1_score(all_targets, all_preds, zero_division=0)

    print(f"Epoch {epoch+1}/{epochs} "
          f"- Train Loss: {avg_train_loss:.4f} "
          f"- Val Loss: {avg_val_loss:.4f} "
          f"- Acc: {acc:.4f} "
          f"- F1: {f1:.4f}")
